## Imports

In [71]:
import pandas as pd

from requests import get
from bs4 import BeautifulSoup

from urllib.parse import quote_plus
import sqlalchemy

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import datetime
import time

## Helper Functions

In [2]:
def to_csv(df_name,index=True):
    df=eval(df_name)
    file_name="outputs/{}_{}.csv".format(df_name,
                                         datetime.datetime.now().strftime("%Y%m%d_%H%M%S"))
    df.to_csv(file_name,index=index)
    print("{} has been output".format(file_name))
    

## Create Engine

In [3]:
def ms_create_engine(db,server="MSI\TEST_LF1",mode="PROD"):
    if mode == "TEST": db="TEST_{}".format(db)
    
    conn_str=(r'Driver={SQL Server};'
              r'Server='+server + ';'
              r'Database=' + db +';'
              r'Trusted_Connection=yes;')
    
    quoted_conn_str=quote_plus(conn_str)
    engine=sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(quoted_conn_str))
    
    return engine

ent_engine=ms_create_engine("SPORTSBOOK")
#prod_ent_engine=ms_create_engine("ENT_Warehouse","TORSQL")

## Scrape Functions

In [4]:

def compile_handicap(input_handicap):
    handicap_list=input_handicap
    output_hc_list=[]
    temp_hc_list=[]
    temp_hc_dict={}
    for handicap in handicap_list:
        hc=handicap.lower()

        if "+" in hc:
            temp_hc_dict["line_type"]="favorite"
            temp_hc_dict["line"]=hc.split("+")[1]
            output_hc_list.append(temp_hc_dict)
            temp_hc_dict={}
        elif "-" in hc:
            temp_hc_dict["line_type"]="underdog"
            temp_hc_dict["line"]=hc.split("-")[1]
            output_hc_list.append(temp_hc_dict)
            temp_hc_dict={}
        elif "o" in hc:
            temp_hc_dict["line_type"]="over"
            #temp_hc_dict["line"]=hc.split("o")[1]

        elif "u" in hc:
            temp_hc_dict["line_type"]="under"
            #temp_hc_dict["line"]=hc.split("u")[1]
        else:
            temp_hc_dict["line"]=hc
            output_hc_list.append(temp_hc_dict)
            temp_hc_dict={}
    return output_hc_list
    

In [5]:
#Start Chromedrivert
def init_chromedriver():
    #ser=Service("C://Users//fermo//Downloads//chromedriver_win32")
    ser="C://Users//fermo//Downloads//chromedriver_win32//chromedriver"
    op=webdriver.ChromeOptions()
    d=webdriver.Chrome(ser,options=op)
    return d

d=init_chromedriver()

In [104]:
to_csv("bb_df")

outputs/bb_df_20221107_183531.csv has been output


In [89]:
bb_df.shape

(1244, 12)

## Bodog Functions

In [111]:
def scrape_bodog_basketball(d):
        
        
    bodog_columns=['timestamp',
'hero',
'villain',
"home_flag",
'market',
'line',
'line_type',
'price',
'match_id',
'league',
'sport',
'hero_score',
'villain_score',
'line_key']

    
    
    target_url="https://www.bodog.eu/sports/basketball"
    d.get(target_url)

    time.sleep(10)

    #bodog_open_coupon_groups(d,0)
    
    coupon_elem=d.find_elements(By.CLASS_NAME,'multi3')
    print(len(coupon_elem))
    bet_list=[]
    hist_market_elem=0
    for ic,coupon in enumerate(coupon_elem[:]):
        #print(c.get_attribute("outerHTML"))
        
        temp_dict_list,hist_market_elem=scrape_bodog_coupon(coupon,ic,hist_market_elem)
        if temp_dict_list: bet_list=bet_list + temp_dict_list

    bet_df=pd.DataFrame(bet_list)
    bet_df["line_key"]=bet_df["match"]
    bet_df=bet_df[bodog_columns]
    
    print("Output Bet Dataframe")
    return bet_df




def scrape_bodog_coupon(coupon_elem,counter,hist_market_elem):
    
    meta_dict=bodog_get_match_meta_data(coupon_elem)

    temp_dict_list=[]
    #Get market label headers. If no header for coupon, use last header
    try:
        #if counter>0: hist_market_elem=market_elem
        market_elem=coupon_elem.find_element(By.CLASS_NAME,"market-header")
        hist_market_elem=market_elem
    except Exception as e:
        market_elem=hist_market_elem
    try: 
    
        market_split=market_elem.get_attribute("innerText").split("\n")
    except:
        return 0, hist_market_elem

    bet_price=coupon_elem.find_elements(By.CLASS_NAME,'bet-price')
    handicap_elem=coupon_elem.find_elements(By.CLASS_NAME,"market-line")
    handicap=[x.get_attribute("innerHTML") for x in handicap_elem]
    handicap_list=compile_handicap(handicap)

    
    hc_counter=0
    for im, market in enumerate(market_split):
        
        for ib,bet in enumerate(bet_price[im*2:im*2+2]):
            price=bet.get_attribute("innerText")
            temp_dict=meta_dict.copy()
            
            
            temp_dict["home_flag"]=ib%2
            temp_dict["hero"]=meta_dict["comp_names"][ib%2]
            temp_dict["villain"]=meta_dict["comp_names"][(ib+1)%2]
            temp_dict["market"]=market
            if "(" in price:
                temp_dict["line"]=handicap_list[hc_counter]["line"]
                temp_dict["line_type"]=handicap_list[hc_counter]["line_type"]
            else:
                temp_dict["line"]=""
                temp_dict["line_type"]=""

            temp_dict["price"]=price.replace("(","").replace(")","")
            try:
                temp_dict["hero_score"]=meta_dict["results_value"][ib%2]
                temp_dict["villain_score"]=meta_dict["results_value"][(ib+1)%2]
            except: 
                temp_dict["hero_score"]=""
                temp_dict["villain_score"]=""
                
            temp_dict_list.append(temp_dict)

    return  temp_dict_list,hist_market_elem    


def bodog_get_match_meta_data(coupon_elem):
    meta_dict={}
    try:
        match_id_elem=coupon_elem.find_element(By.TAG_NAME,"a")
    except Exception as e:
        print(e)
        return 0,hist_market_elem
    match_id_split=match_id_elem.get_attribute("href").split("/")
    meta_dict["match_id"]=match_id_split[-1]
    print(meta_dict["match_id"])
    meta_dict["league"]=match_id_split[-2]
    meta_dict["sport"]=match_id_split[4]

    comp_elem=coupon_elem.find_elements(By.CLASS_NAME,"competitor-name")
    meta_dict["comp_names"]=[x.get_attribute("innerText") for x in comp_elem]
    results_elem=coupon_elem.find_elements(By.CLASS_NAME,"score-nr")

    meta_dict["results_value"]=[x.get_attribute("innerText") for x in results_elem][:2]
    meta_dict["timestamp"]=datetime.datetime.now()
    return meta_dict
    #def bodog_scrape_market(d):
    

#to_csv("bet_df")

In [106]:
def bodog_open_coupon_groups(d,counter):
    
    try:
        print("showing more")
        show_more_elem=d.find_element(By.ID,"showMore")
        show_more_elem.click()
        time.sleep(4)
        if d.find_element(By.ID,"showMore"):
            bodog_open_coupon_groups(d,counter+1)
            return 0
    except Exception as e:
        print(e)
        print("no show more button")
        pass
        
    header_elem=d.find_elements(By.CLASS_NAME,"league-header-collapsible--collapsed")

    for h in header_elem:
        try:
            h.click()
        except:
            continue
    print("opened")
    return d

## Scheduled Code

In [112]:
d=init_chromedriver()
bb_df=scrape_bodog_basketball(d)
#bb_df.to_sql("fact_lines",ent_engine,if_exists='append',index=False)

60
coppin-state-charlotte-202211071800
ipfw-michigan-22-202211071830
la-salle-villanova-16-202211071830
colonias-gold-basketball-libertad-asuncion-202211071730
olimpia-kings-deportivo-san-jose-202211071730
mil-bucks-active-pg-bos-celtics-shinigami-202211071822


StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=107.0.5304.88)


In [ ]:
bb_df.head()